In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json

In [3]:
def fetch_bycode_with_retry(retries=5, delay=5):
    """Gửi request với retry khi lỗi."""
    for attempt in range(retries):
        try:
            url = "https://trade.ec.europa.eu/access-to-markets/api/v2/nomenclature/products"
            params = {
                "country": "AT",
                "lang": "EN"
            }

            response = requests.get(url, params=params, timeout=60)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return response
        except requests.RequestException as e:
            print(f"Request error: {e} - Retrying {attempt+1}/{retries}...")
            time.sleep(delay)
    return None

In [4]:
def fetch_byid_with_retry(id, retries=5, delay=5):
    """Gửi request với retry khi lỗi."""
    for attempt in range(retries):
        try:
            url = "https://trade.ec.europa.eu/access-to-markets/api/v2/nomenclature/products"
            params = {
                "parent": id,
                "country": "AT",
                "lang": "EN"
            }

            response = requests.get(url,params=params,  timeout=60)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return response
        except requests.RequestException as e:
            print(f"Request error: {e} - Retrying {attempt+1}/{retries}...")
            time.sleep(delay)
    return None

In [5]:
section_info = fetch_bycode_with_retry().json()
section_df = pd.DataFrame(section_info)
section_df['index'] = section_df.index
section_df['crawled'] = ''
# df = section_df.copy()
# df.to_csv('section_info.csv', index=False)
# df

In [8]:
df = pd.read_csv('at_hs_info.csv')
df

,Unnamed: 0.1,Unnamed: 0,id,code,description,hasChildren,section,index,crawled,parent_id,selectable,intervalMin,intervalMax,error_message
0,0,0.0,-1,NaN,Live animals; animal products,True,"{'description': 'SECTION I', 'chapterFrom': '0...",0,Done,NaN,NaN,NaN,NaN,NaN
1,1,1.0,-2,NaN,Vegetable products,True,"{'description': 'SECTION II', 'chapterFrom': '...",1,Done,NaN,NaN,NaN,NaN,NaN
2,2,2.0,-3,NaN,Animal or vegetable fats and oils and their cl...,True,"{'description': 'SECTION III', 'chapterFrom': ...",2,Done,NaN,NaN,NaN,NaN,NaN
3,3,3.0,-4,NaN,"Prepared foodstuffs; beverages, spirits and vi...",True,"{'description': 'SECTION IV', 'chapterFrom': '...",3,Done,NaN,NaN,NaN,NaN,NaN
4,4,4.0,-5,NaN,Mineral products,True,"{'description': 'SECTION V', 'chapterFrom': '2...",4,Done,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6796,6796,NaN,3847,47073090.0,Other,False,NaN,1,NaN,2347.0,True,NaN,NaN,NaN
6797,6797,NaN,3947,47079010.0,Unsorted,False,NaN,0,NaN,2447.0,True,NaN,NaN,NaN
6798,6798,NaN,4047,47079090.0,Sorted,False,NaN,1,NaN,2447.0,True,NaN,NaN,NaN
6799,6799,NaN,19248,48024010.0,Not containing fibres obtained by a mechanical...,False,NaN,0,NaN,3448.0,True,NaN,NaN,NaN


In [ ]:
df = pd.read_csv('at_hs_info.csv')
while ((df['hasChildren'] == True) & ((df['crawled'] == '') | pd.isna(df['crawled']) | (df['crawled'] == 'Error'))).any():
    for i in df.index:
        if df.loc[i, 'hasChildren'] and (df.loc[i, 'crawled'] == '' or pd.isna(df.loc[i, 'crawled']) or df.loc[i, 'crawled'] == 'Error'):
            id = df.loc[i, 'id']
            attempts = 0
            success = False
            error_occurred = False  # Biến để kiểm soát lỗi

            while attempts < 5 and not success:
                try:
                    print(f"Fetching index: {i} id: {id}... (Attempt {attempts}/5)")
                    
                    response = fetch_byid_with_retry(id)  
                    if response and response.status_code == 200:
                        df2 = pd.DataFrame(response.json())
                        df2['index'] = df2.index
                        df2['parent_id'] = id
                        print(df2)
                        
                        df = pd.concat([df, df2], ignore_index=True) 
                        df.loc[i, 'crawled'] = 'Done'
                        print(f"✅ Done fetching index: {i} id: {id}")
                        success = True  
                    else:
                        raise Exception(f"Failed request, status code: {response.status_code}")
                
                except Exception as e:
                    attempts += 1
                    error_message = str(e)
                    print(f"❌ Error fetching index: {i} id: {id}... (Attempt {attempts}/5) - {e}")
                    if attempts == 5: 
                        error_occurred = True

            if error_occurred: 
                df.loc[i, 'crawled'] = 'Error'
                df.loc[i, 'error_message'] = error_message
                print("⏳ Encountered multiple errors. Sleeping for 5 minutes...")
                time.sleep(60)  
                break
        df.to_csv('at_hs_info.csv', index=True)


KeyboardInterrupt: 